Check Nvidia Drivers, make sure you have chosen GPU runtime from Runtime > Change runtime type > GPU

In [1]:
!nvidia-smi

Sat Dec 11 13:29:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Step 0: Install OpenNMT-py**

In [2]:
!pip install --upgrade pip
!pip install OpenNMT-py

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 216 kB 5.3 MB/s            
     |████████████████████████████████| 16.3 MB 37.5 MB/s            
     |████████████████████████████████| 56 kB 4.2 MB/s             
     |████████████████████████████████| 73 kB 1.7 MB/s             
     |████████████████████████████████| 1.2 MB 38.4 MB/s            
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [3]:
!head -n 3 tgt-train.txt

@some users congratulate you for celebrating British kid singers Sophia Grace's and Rosie's 1st anniversary visit of your show
@Lokardi_ rat we should a lot more for that, by this evi people nothing will happen #ForTheSakeOfHumanity
@Slimswami ehi, this is what i'm expecting you to understand, people invest everything in this isnt it.


We need to build a YAML configuration file to specify the data that will be used:

In [12]:
%%bash
cat <<EOF > config.yaml

## Where the samples will be written
save_data: opennmt_codemixed/example
## Where the vocab(s) will be written
src_vocab: opennmt_codemixed/example.vocab.src
tgt_vocab: opennmt_codemixed/example.vocab.tgt
# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: opennmt_codemixed/src-train.txt
        path_tgt: opennmt_codemixed/tgt-train.txt
    valid:
        path_src: opennmt_codemixed/src-val.txt
        path_tgt: opennmt_codemixed/tgt-val.txt

EOF

From this configuration, we can build the vocab(s), that will be necessary to train the model:

In [13]:
!onmt_build_vocab -config config.yaml -n_sample 20000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-12-11 13:37:43,866 INFO] Counter vocab from 20000 samples.
[2021-12-11 13:37:43,866 INFO] Build vocab on 20000 transformed examples/corpus.
[2021-12-11 13:37:43,879 INFO] corpus_1's transforms: TransformPipe()
[2021-12-11 13:37:43,881 WARNING] Empty line exists in corpus_1#62.
[2021-12-11 13:37:43,882 WARNING] Empty line exists in corpus_1#63.
[2021-12-11 13:37:43,882 WARNING] Empty line exists in corpus_1#104.
[2021-12-11 13:37:43,882 WARNING] Empty line exists in corpus_1#105.
[2021-12-11 13:37:43,883 WARNING] Empty line exists in corpus_1#127.
[2021-12-11 13:37:43,883 WARNING] Empty line exists in corpus_1#129.
[2021-12-11 13:37:43,883 WARNING] Empty line exists in corpus_1#150.
[2021-12-11 13:37:43,884 WARNING] Empty line exists in corpus_1#152.
[2021-12-11 13:37:43,884 WARNING] Empty line exists in corpus_1#172.
[2021-12-11 13:37:43,885 WARNING] Empty line exists in corpus_1#199.
[2021-12-11 13:37:43,885 W

Notes:

-n_sample is required here – it represents the number of lines sampled from each corpus to build the vocab.

This configuration is the simplest possible, without any tokenization or other transforms. See other example configurations for more complex pipelines.

**Step 2: Train the model**

To train a model, we need to add the following to the YAML configuration file:

the vocabulary path(s) that will be used: can be that generated by onmt_build_vocab;

training specific parameters.

In [14]:
%%bash 
cat <<EOF >> config.yaml
# Vocabulary files that were just created
src_vocab: opennmt_codemixed/example.vocab.src
tgt_vocab: opennmt_codemixed/example.vocab.tgt

# Where to save the checkpoints
save_model: opennmt_codemixed/run/model
save_checkpoint_steps: 2000
keep_checkpoint: 10
seed: 3435
train_steps: 2000
valid_steps: 1000
warmup_steps: 500
report_every: 500

decoder_type: transformer
encoder_type: transformer
word_vec_size: 256
rnn_size: 256
layers: 4
transformer_ff: 512
heads: 4

accum_count: 8
optim: adam
adam_beta1: 0.9
adam_beta2: 0.998
decay_method: noam
learning_rate: 2.0
max_grad_norm: 0.0

batch_size: 512
batch_type: tokens
normalization: tokens
dropout: 0.1
label_smoothing: 0.1

max_generator_batches: 2

param_init: 0.0
param_init_glorot: 'true'
position_encoding: 'true'

world_size: 1
gpu_ranks: [0]
EOF

Then you can simply run:

In [15]:
!onmt_train -config config.yaml

Streaming output truncated to the last 5000 lines.
[2021-12-11 13:54:42,870 WARNING] Empty line exists in corpus_1#7435.
[2021-12-11 13:54:42,870 WARNING] Empty line exists in corpus_1#7437.
[2021-12-11 13:54:42,870 WARNING] Empty line exists in corpus_1#7441.
[2021-12-11 13:54:42,871 WARNING] Empty line exists in corpus_1#7457.
[2021-12-11 13:54:42,871 WARNING] Empty line exists in corpus_1#7486.
[2021-12-11 13:54:42,872 WARNING] Empty line exists in corpus_1#7604.
[2021-12-11 13:54:50,548 WARNING] Empty line exists in corpus_1#7714.
[2021-12-11 13:54:50,548 WARNING] Empty line exists in corpus_1#7722.
[2021-12-11 13:54:50,549 WARNING] Empty line exists in corpus_1#7774.
[2021-12-11 13:54:50,549 WARNING] Empty line exists in corpus_1#7855.
[2021-12-11 13:54:50,549 WARNING] Empty line exists in corpus_1#7857.
[2021-12-11 13:54:50,550 WARNING] Empty line exists in corpus_1#7909.
[2021-12-11 13:54:50,550 WARNING] Empty line exists in corpus_1#7947.
[2021-12-11 13:54:50,555 WARNING] Empty

This configuration will run the default model, which consists of a 2-layer LSTM with 500 hidden units on both the encoder and decoder. It will run on a single GPU (world_size 1 & gpu_ranks [0]).

Before the training process actually starts, the *.vocab.pt together with *.transforms.pt can be dumped to -save_data with configurations specified in -config yaml file by enabling the -dump_fields and -dump_transforms flags. It is also possible to generate transformed samples to simplify any potentially required visual inspection. The number of sample lines to dump per corpus is set with the -n_sample flag.

For more advanded models and parameters, see other example configurations or the FAQ.

**Step 3: Translate**

In [19]:
!onmt_translate -model opennmt_codemixed/run/model_step_2000.pt -src opennmt_codemixed/src-train.txt -output opennmt_codemixed/pred_1000.txt -gpu 0 -verbose

Streaming output truncated to the last 5000 lines.
SENT 2560: ['Desh', 'ke', 'saare', 'judge', 'bhaitard', 'hai', 'kya', '??https://twitter.com/aajtak/status/626259408239112192Â', 'â€¦']
PRED 2560: )
PRED SCORE: -4.7232

[2021-12-11 14:14:07,785 INFO] 
SENT 2561: ['@rsrajat365', 'bahut', 'drama', 'karta', 'hai', 'yaar']
PRED 2561: )
PRED SCORE: -4.8324

[2021-12-11 14:14:07,786 INFO] 
SENT 2562: ['"Waise', '#JNU', 'me', 'aaj', 'kal', '#KulbhushanJadhav', 'ke', 'liye', 'naare', 'lagte', 'hai', 'kya?', 'Did', 'any', 'one', 'return', 'his/her', '#award', 'for', 'him?']
PRED 2562: )
PRED SCORE: -4.8136

[2021-12-11 14:14:07,786 INFO] 
SENT 2563: ['aur', 'kuch', 'dost', 'guddu', 'k', 'face', 'expression', 'dekhne', 'aate', 'hai', 'yahan', ':d', ':d', ':d']
PRED 2563: )
PRED SCORE: -4.7672

[2021-12-11 14:14:07,786 INFO] 
SENT 2564: ['@drpoonam', '@runjhunmehrotra', 'hadh', 'hoti', 'hai', 'yaar', ":'("]
PRED 2564: )
PRED SCORE: -4.6421

[2021-12-11 14:14:07,786 INFO] 
SENT 2565: ['@iNeelSoni

Now you have a model which you can use to predict on new data. We do this by running beam search. This will output predictions into toy-ende/pred_1000.txt.

Note:

The predictions are going to be quite terrible, as the demo dataset is small. Try running on some larger datasets! For example you can download millions of parallel sentences for translation or summarization.